____

# Trabajo Práctico Ciencia de Datos: Telco Churn

____


__Universidad Tecnológica Nacional, Buenos Aires__<br/>
__Ingeniería Industrial__<br/>
__Cátedra de Ciencia de Datos - Cluster AI__<br/>
__Elaborado por: Gaspar Rivollier__<br/>
__2023__<br/>
____

# 1. Set up de notebook

In [153]:
#Importamos librerías para EDA y Machine Learning en python
# importamos las librerías necesarias para trabajar.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importamos librerias de scikit learn
from sklearn import svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification
from sklearn import preprocessing

# 2. Import del dataset y pre-processing

In [154]:
#Preparamos notebook para trabajar localmente o en GoogleCollab
# Verificamos si estamos en Colab
var_google_colab = 'google.colab' in str(get_ipython())
print(var_google_colab)
if var_google_colab:
  #Montamos nuestro G.Drive
  from google.colab import drive
  drive.mount('/content/gdrive',force_remount=True)
  # Direccion root donde está la jupyter-notebook
  root_path = "/content/gdrive/MyDrive/UTN/TP Cs Datos/telco_churn_clusterai.csv"
  # Direccion donde guardaremos las imagenes
  plot_path = root_path
else:
  # Si, no estamos en google colab, es que estamos corriendo la
  # en local.
  root_path = ""
  plot_path = root_path

True
Mounted at /content/gdrive


In [155]:
#Importamos
clients_df = pd.read_csv(root_path)

In [156]:
#Verificamos dimensiones y head del df
print(clients_df.shape)
clients_df.head(5)

(7043, 22)


,Unnamed: 0,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,0,7590-VHVEG,Female,NaN,Yes,No,1.0,No,No phone service,DSL,...,No,No,No,No,Month-to-month,NaN,Electronic check,29.85,29.85,No
1,1,5575-GNVDE,Male,0.0,No,No,34.0,Yes,No,NaN,...,Yes,No,No,No,One year,No,Mailed check,NaN,1889.5,No
2,2,3668-QPYBK,Male,0.0,No,No,2.0,NaN,No,NaN,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,3,7795-CFOCW,Male,0.0,No,No,NaN,No,NaN,NaN,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,4,9237-HQITU,Female,NaN,NaN,NaN,2.0,Yes,No,Fiber optic,...,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,151.65,Yes


In [157]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        7043 non-null   int64  
 1   customerID        7043 non-null   object 
 2   gender            6127 non-null   object 
 3   SeniorCitizen     6127 non-null   float64
 4   Partner           6127 non-null   object 
 5   Dependents        6127 non-null   object 
 6   tenure            6127 non-null   float64
 7   PhoneService      6127 non-null   object 
 8   MultipleLines     6127 non-null   object 
 9   InternetService   6127 non-null   object 
 10  OnlineSecurity    6127 non-null   object 
 11  OnlineBackup      6127 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          5846 non-null   object 


In [158]:
#Calculamos cantidad de NaNs
clients_df.isnull().sum()

Unnamed: 0             0
customerID             0
gender               916
SeniorCitizen        916
Partner              916
Dependents           916
tenure               916
PhoneService         916
MultipleLines        916
InternetService      916
OnlineSecurity       916
OnlineBackup         916
DeviceProtection       0
TechSupport            0
StreamingTV            0
StreamingMovies        0
Contract            1197
PaperlessBilling    1197
PaymentMethod       1197
MonthlyCharges      1197
TotalCharges           0
Churn                  0
dtype: int64

In [159]:
#Verificamos como quedaría el dataset quitando los null
clients_temp = clients_df.dropna()
clients_temp.shape

(847, 22)

In [160]:
#Siendo que se eliminarían 6000 registros de 7000 en caso de quitar todos los nulos, reemplazaremos algunos NaNs por promedios, y removeremos aquellos que consideremos que afecta a la informacion de los datos si se rellena.
#Además quitamos columna customerID y Unnamed ya que no aporta valor al modelo de aprendizaje
clients_df = clients_df.iloc[:,2:]
clients_df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,NaN,Yes,No,1.0,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,NaN,Electronic check,29.85,29.85,No
1,Male,0.0,No,No,34.0,Yes,No,NaN,Yes,No,Yes,No,No,No,One year,No,Mailed check,NaN,1889.5,No
2,Male,0.0,No,No,2.0,NaN,No,NaN,Yes,NaN,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0.0,No,No,NaN,No,NaN,NaN,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,NaN,NaN,NaN,2.0,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,NaN,151.65,Yes


In [161]:
for field in clients_df:
  print(f"#{field}")
  print(clients_df[field].unique())
  print()

#gender
['Female' 'Male' nan]

#SeniorCitizen
[nan  0.  1.]

#Partner
['Yes' 'No' nan]

#Dependents
['No' nan 'Yes']

#tenure
[ 1. 34.  2. nan  8. 22. 10. 28. 62. 13. 16. 58. 49. 25. 69. 52. 71. 21.
 47. 72. 17. 27.  5. 46. 11. 70. 43. 60. 18. 63. 66.  9.  3. 31. 50. 64.
 56. 30. 45.  7. 42. 35. 48. 29. 12. 38. 32. 55. 37. 36. 41.  6.  4. 33.
 23. 57. 61. 65. 20. 53. 14. 15. 40. 59. 24. 44. 19. 54. 51. 67. 68. 26.
  0. 39.]

#PhoneService
['No' 'Yes' nan]

#MultipleLines
['No phone service' 'No' nan 'Yes']

#InternetService
['DSL' nan 'Fiber optic' 'No']

#OnlineSecurity
['No' 'Yes' nan 'No internet service']

#OnlineBackup
['Yes' 'No' nan 'No internet service']

#DeviceProtection
['No' 'Yes' 'No internet service']

#TechSupport
['No' 'Yes' 'No internet service']

#StreamingTV
['No' 'Yes' 'No internet service']

#StreamingMovies
['No' 'Yes' 'No internet service']

#Contract
['Month-to-month' 'One year' nan 'Two year']

#PaperlessBilling
[nan 'No' 'Yes']

#PaymentMethod
['Electronic che

In [162]:
#Basado en estos valores unicos, reemplazaremos por promedios en las columnas MonthlyCharges y tenure
clients_df['MonthlyCharges'] = clients_df['MonthlyCharges'].replace(np.nan, clients_df['MonthlyCharges'].mean())
clients_df['tenure'] = clients_df['tenure'].replace(np.nan, clients_df['tenure'].mean())

In [ ]:
#Para el caso de InternetService, si cualquiera de los campos que contienen la categoría 'No internet Service' vale eso, se puede inferir que este debe valer 'No'. De forma inversa, si el campo internet service vale 'No' se puede inferir que el resto de campos deberá valer 'No internet service'.
for ind in clients_df.index:
  if clients_df['OnlineSecurity'][ind] == 'No internet service' or clients_df['OnlineBackup'][ind] == 'No internet service' or clients_df['DeviceProtection'][ind] == 'No internet service' or clients_df['TechSupport'][ind] == 'No internet service' or clients_df['DeviceProtection'][ind] == 'No internet service' or clients_df['TechSupport'][ind] == 'No internet service' or clients_df['StreamingTV'][ind] == 'No internet service' or clients_df['StreamingMovies'][ind] == 'No internet service' or clients_df['InternetService'][ind] == 'No':
    clients_df['OnlineSecurity'][ind] = 'No'
    clients_df['OnlineBackup'][ind] = 'No'
    clients_df['DeviceProtection'][ind] = 'No'
    clients_df['TechSupport'][ind] = 'No'
    clients_df['DeviceProtection'][ind] = 'No'
    clients_df['TechSupport'][ind] = 'No'
    clients_df['StreamingTV'][ind] = 'No'
    clients_df['StreamingMovies'][ind] = 'No'
    clients_df['InternetService'][ind] = 'No'

In [164]:
#Volvemos a ver unique values
for field in clients_df:
  print(f"#{field}")
  print(clients_df[field].unique())
  print()

#gender
['Female' 'Male' nan]

#SeniorCitizen
[nan  0.  1.]

#Partner
['Yes' 'No' nan]

#Dependents
['No' nan 'Yes']

#tenure
[ 1.         34.          2.         32.53974213  8.         22.
 10.         28.         62.         13.         16.         58.
 49.         25.         69.         52.         71.         21.
 47.         72.         17.         27.          5.         46.
 11.         70.         43.         60.         18.         63.
 66.          9.          3.         31.         50.         64.
 56.         30.         45.          7.         42.         35.
 48.         29.         12.         38.         32.         55.
 37.         36.         41.          6.          4.         33.
 23.         57.         61.         65.         20.         53.
 14.         15.         40.         59.         24.         44.
 19.         54.         51.         67.         68.         26.
  0.         39.        ]

#PhoneService
['No' 'Yes' nan]

#MultipleLines
['No phone service' 

In [165]:
#Calculamos cantidad de NaNs
clients_df.isnull().sum()

gender               916
SeniorCitizen        916
Partner              916
Dependents           916
tenure                 0
PhoneService         916
MultipleLines        916
InternetService      733
OnlineSecurity       722
OnlineBackup         738
DeviceProtection       0
TechSupport            0
StreamingTV            0
StreamingMovies        0
Contract            1197
PaperlessBilling    1197
PaymentMethod       1197
MonthlyCharges         0
TotalCharges           0
Churn                  0
dtype: int64

In [166]:
#Verificamos nuevamente como quedaría df si eliminamos NaNs
#Verificamos como quedaría el dataset quitando los null
clients_temp = clients_df.dropna()
clients_temp.shape

(1276, 20)

In [167]:
#Procesados los NaNs, guardamos en base clients_df
clients_df = clients_temp

In [168]:
#Debemos convertir ahora todos los valores a valores numericos. Para las columnas booleanas (Yes/No) reemplazaremos por 1 y 0, respectivamente. Para variables categoricas, se crearan variables dummies.
clients_df.replace('Yes',1,inplace=True)
clients_df.replace('No',0,inplace=True)

<ipython-input-168-a4e77d2efb4d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clients_df.replace('Yes',1,inplace=True)


In [169]:
clients_df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
10,Male,0.0,1,1,13.000000,1,0,DSL,1,0,0,0,0,0,Month-to-month,1,Mailed check,49.950000,587.45,0
18,Female,0.0,1,1,10.000000,1,0,DSL,0,0,1,1,0,0,Month-to-month,0,Credit card (automatic),64.658655,528.35,1
21,Male,0.0,1,0,32.539742,1,0,0,0,0,0,0,0,0,One year,0,Bank transfer (automatic),19.800000,202.25,0


In [170]:
#Generamos variables dummies para columnas InternetService, Contract, gender y PaymentMethod
## Genero las dummies para la variable InternetService, dentro de la variable internet_dummies.
internet_dummies = pd.get_dummies(data=clients_df.loc[:,'InternetService'])
## Genero las dummies para la variable Contract, dentro de la variable contract_dummies.
contract_dummies = pd.get_dummies(data=clients_df.loc[:,'Contract'])
## Genero las dummies para la variable PaymentMethod, dentro de la variable payment_dummies.
payment_dummies = pd.get_dummies(data=clients_df.loc[:,'PaymentMethod'])
## Genero las dummies para la variable PaymentMethod, dentro de la variable gender_dummies.
gender_dummies = pd.get_dummies(data=clients_df.loc[:,'gender'])

In [171]:
#Head de dummies
internet_dummies.head(3)

,0,DSL,Fiber optic
10,0,1,0
18,0,1,0
21,1,0,0


In [172]:
#Generamos variables dummies para columnas InternetService, Contract, gender y PaymentMethod
## Genero las dummies para la variable InternetService, dentro de la variable internet_dummies.
internet_dummies = pd.get_dummies(data=clients_df.loc[:,'InternetService'])
## Genero las dummies para la variable Contract, dentro de la variable contract_dummies.
contract_dummies = pd.get_dummies(data=clients_df.loc[:,'Contract'])
## Genero las dummies para la variable PaymentMethod, dentro de la variable payment_dummies.
payment_dummies = pd.get_dummies(data=clients_df.loc[:,'PaymentMethod'])
## Genero las dummies para la variable PaymentMethod, dentro de la variable gender_dummies.
gender_dummies = pd.get_dummies(data=clients_df.loc[:,'gender'])

In [173]:
#Head de InternetService dummy
internet_dummies.rename(columns={0: "NoInternet"},inplace=True)
internet_dummies.head(3)

,NoInternet,DSL,Fiber optic
10,0,1,0
18,0,1,0
21,1,0,0


In [174]:
#Head de dummies
contract_dummies.head(3)

,Month-to-month,One year,Two year
10,1,0,0
18,1,0,0
21,0,1,0


In [175]:
#Head de dummies
payment_dummies.head(3)

,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check
10,0,0,0,1
18,0,1,0,0
21,1,0,0,0


In [176]:
#Head de dummies
gender_dummies.head(3)

,Female,Male
10,0,1
18,1,0
21,0,1


In [177]:
#Hacemos join de variables dummy con data ser original
clients_df = clients_df.join(internet_dummies)
clients_df = clients_df.join(contract_dummies)
clients_df = clients_df.join(payment_dummies)
clients_df = clients_df.join(gender_dummies)

In [178]:
clients_df.head(5)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,Fiber optic,Month-to-month,One year,Two year,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check,Female,Male
10,Male,0.0,1,1,13.000000,1,0,DSL,1,0,...,0,1,0,0,0,0,0,1,0,1
18,Female,0.0,1,1,10.000000,1,0,DSL,0,0,...,0,1,0,0,0,1,0,0,1,0
21,Male,0.0,1,0,32.539742,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
23,Female,0.0,1,0,58.000000,1,1,DSL,0,1,...,0,0,0,1,0,1,0,0,1,0
25,Female,0.0,0,0,32.539742,1,0,DSL,1,1,...,0,1,0,0,1,0,0,0,1,0


In [179]:
#Dropear columnas que pasamos a dummy.
clients_df = clients_df.drop('InternetService',axis=1)

In [180]:
clients_df = clients_df.drop('Contract',axis=1)
clients_df = clients_df.drop('PaymentMethod', axis=1)
clients_df = clients_df.drop('gender', axis=1)

In [181]:
clients_df.head(5)

,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,...,Fiber optic,Month-to-month,One year,Two year,Bank transfer (automatic),Credit card (automatic),Electronic check,Mailed check,Female,Male
10,0.0,1,1,13.000000,1,0,1,0,0,0,...,0,1,0,0,0,0,0,1,0,1
18,0.0,1,1,10.000000,1,0,0,0,1,1,...,0,1,0,0,0,1,0,0,1,0
21,0.0,1,0,32.539742,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,1
23,0.0,1,0,58.000000,1,1,0,1,0,1,...,0,0,0,1,0,1,0,0,1,0
25,0.0,0,0,32.539742,1,0,1,1,0,0,...,0,1,0,0,1,0,0,0,1,0


In [182]:
#Resumimos informacion del dataframe luego de las transformaciones
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1276 entries, 10 to 7033
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   SeniorCitizen              1276 non-null   float64
 1   Partner                    1276 non-null   int64  
 2   Dependents                 1276 non-null   int64  
 3   tenure                     1276 non-null   float64
 4   PhoneService               1276 non-null   int64  
 5   MultipleLines              1276 non-null   object 
 6   OnlineSecurity             1276 non-null   int64  
 7   OnlineBackup               1276 non-null   int64  
 8   DeviceProtection           1276 non-null   int64  
 9   TechSupport                1276 non-null   int64  
 10  StreamingTV                1276 non-null   int64  
 11  StreamingMovies            1276 non-null   int64  
 12  PaperlessBilling           1276 non-null   int64  
 13  MonthlyCharges             1276 non-null   floa

In [183]:
#Trabamos el dataset para que variables tipo objeto se convieran en int64. Para ello aseguramos que unique values solo sean valores int.
#MultipleLines
print(clients_df['MultipleLines'].unique())
#Reemplazamos 'No phone service' por 0 (no).
clients_df['MultipleLines'].replace('No phone service',inplace=True)
print(clients_df['MultipleLines'].unique())

[0 1 'No phone service']
[0 1]


In [184]:
object_features = ['MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
                 'TechSupport', 'StreamingTV', 'StreamingMovies']

# Print unique values for each object field
for field in object_features:
    print(f"#{field}")
    print(clients_df[field].unique())
    print()

#MultipleLines
[0 1]

#OnlineSecurity
[1 0]

#OnlineBackup
[0 1]

#DeviceProtection
[0 1]

#TechSupport
[0 1]

#StreamingTV
[0 1]

#StreamingMovies
[0 1]



In [185]:
#Como ya no tenemos mas valores fuera de 0 y 1, convertimos columna a int
clients_df[object_features] = clients_df[object_features].astype('int64')

In [187]:
#Pasamos a TotalCharges
print(clients_df['TotalCharges'].unique())

['587.45' '528.35' '202.25' ... '263.05' '75.75' '2625.25']


In [188]:
#Este caso corresponde a un valor que debería ser float. Al intentar convertir tenemos problemas por valores ' ' que deberían ser NaN. Reemplazamos por el promedio.
#Reemplazamos empty con NaN
clients_df['TotalCharges'] = pd.to_numeric(clients_df['TotalCharges'].replace(' ', np.nan), errors='coerce')

# Calculo media
mean_total_charges = clients_df['TotalCharges'].mean()

# Replace NaN values with the mean
clients_df['TotalCharges'].fillna(mean_total_charges, inplace=True)

In [189]:
#Se ajusta tipo de dato a float 
clients_df['TotalCharges'] = clients_df['TotalCharges'].astype('float64')

In [190]:
#Exportamos base pre procesada como csv para utilizarla en siguiente notebook
clients_df.to_csv('/content/gdrive/MyDrive/UTN/TP Cs Datos/preprocessed_telco_churn_clusterai.csv')